<a href="https://colab.research.google.com/github/sincendz/mineracao-de-dados/blob/main/02.E0-Exercicio-Classificacao-de-Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/paulotguerra/QXD0178/blob/main/02.E0-Exercicio-Classificacao-de-dados.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## QXD0178 - Mineração de Dados
# Classificação de dados

**Professor:** Paulo de Tarso Guerra Oliveira ([paulodetarso@ufc.br](mailto:paulodetarso@ufc.br))


# Lista de Exercícios: Classificação de dados

Nesta lista de exercícios, você explorará a aplicação de métodos de aprendizado de máquina para realizar tarefas de classificação de dados. Você usará a base de dados [Food choices: College students' food and cooking preferences](https://www.kaggle.com/datasets/borapajo/food-choices?select=food_coded.csv) e avaliará vários algoritmos de classificação para determinar sua eficácia. O objetivo é entender como diferentes métodos de aprendizado de máquina se comportam em relação à acurácia na classificação de dados.

O exercício será dividido em várias etapas:

1. **Pré-processamento dos dados:**
   - Descreva brevemente o conjunto de dados   
   - Limpe o conjunto de dados, tratando valores ausentes, removendo duplicatas e realizando transformações necessárias.
   - Caso você use os dados pré-processados na lista anterior, faça um breve descritivo dos principais ajustes.
   - Codifique variáveis categóricas, se necessário, para que possam ser utilizadas em algoritmos de aprendizado de máquina.
   - Cria a coluna `self_perception_overweight` com valor: `True` se a coluna `self_perception_weight` tem valor 4 ou 5; e `False`, caso contrário.
   - Remova a coluna `self_perception_weight` do conjunto de dados.
2. **Divisão do conjunto de dados:**
   - Divida o conjunto de dados em um conjunto de treinamento e um conjunto de teste para avaliar o desempenho dos algoritmos.
   - O mesmo conjunto de teste deve ser usado por todos os algoritmos analizados e nenhum dado deste pode ser usado na fase de treinamento.
   - O atributo alvo (*rótulo*) da classificação será o campo `self_perception_overweight`.   
3. **Seleção de algoritmos de classificação:**
   - Selecione uma variedade de algoritmos de aprendizado de máquina para testar na tarefa de classificação.   
   - Sua seleção deve conter, no mínimo, os seguintes métodos: Naive Bayes, k-Nearest Neighbors, Support Vector Machine (Linear/RBF), Decision Trees, Random Forest, Multilayer Perceptron.
   - Descreva brevemente como funciona cada algoritmo selecionado.
4. **Treinamento e avaliação:**
   - Treine os algoritmos de classificação usando todo o conjunto de treinamento.
   - Avalie o desempenho de cada algoritmo no conjunto de teste usando métricas como acurácia, precisão, recall e F1-score.
   - Repita a análise treinando os algoritmos com validação cruzada.
   - Repita a análise realizando ajuste de hiperparâmetros.
5. **Análise dos resultados:**
   - Prepare um texto que descreva os resultados obtidos e faça uma análise crítica destes resultados.
   - Compare o desempenho dos diferentes algoritmos e explique por que alguns apresentaram resultados mais adequados que outros.
   
Documente todas as etapas em um arquivo Jupyter Notebook (`.ipynb`) que inclua as análises, o código e as justificativas. Lembre-se de que é fundamental justificar todas as decisões tomadas ao longo do processo e documentar as análises de forma clara e concisa. Este trabalho tem como objetivo proporcionar uma compreensão prática da seleção e avaliação de algoritmos de classificação em cenários de aprendizado supervisionado.

Envie seu Jupyter Notebook até a data de entrega especificada nesta tarefa.

## Solução


### Pré-processamento dos dados

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report






df = pd.read_csv("https://raw.githubusercontent.com/She-Codes-Now/Intro-to-Data-Science-with-R/master/food_coded.csv")
#df

In [2]:
def is_number(x):
    try:
        float(x)
        return True
    except ValueError:
        return False

#Pega um número de dentro de uma string
def extrair_numeros(string):
    lista = re.findall(r'\d+\.\d+|\d+', string)
    if len(lista) == 0:
        return np.nan
    else:
        return float(lista[0])
    return lista

# Função para converter a coluna para valores numéricos
def to_numeric(df):
    return df.map(lambda x: float(x) if is_number(x) else extrair_numeros(x))

def to_lower_case(df):
    return df.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

def replace_slash_with_comma(df):
    return df.apply(lambda x: x.str.replace('/', ',') if x.dtype == 'object' else x)
def replace_r_with_comma(df):
    return df.apply(lambda x: x.str.replace('\r', ',') if x.dtype == 'object' else x)

In [3]:
gpa = ['GPA']
weight = ['weight']


# Criando o transformer para a coluna GPA
gpa_transformer = Pipeline(steps=[
    ('to_numeric', FunctionTransformer(to_numeric)),
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
]).set_output(transform='pandas')


weight_transformer = Pipeline(steps=[
    ('to_numeric', FunctionTransformer(to_numeric)),
    ('imputer', SimpleImputer(strategy='mean')),
],).set_output(transform='pandas')


preprocessing = ColumnTransformer(transformers=[
    ('GPA', gpa_transformer, ['GPA']),
    ('weight', weight_transformer, ['weight']),
], remainder='passthrough').set_output(transform='pandas')

df = preprocessing.fit_transform(df)


In [4]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Identificar as colunas do tipo object (geralmente colunas de texto ou categóricas)
object_columns = df.select_dtypes(include=['object']).columns.tolist()


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])

string_transformer = Pipeline(steps=[
    ('to_lower_case', FunctionTransformer(to_lower_case)),
    ('replace_slash_with_comma', FunctionTransformer(replace_slash_with_comma)),
    ('replace_r_with_comma', FunctionTransformer(replace_r_with_comma)),
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Usando ColumnTransformer para aplicar a transformação na coluna GPA
preprocessing = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, numeric_columns),  # Aplica a transformação nas colunas numéricas
    ('object', string_transformer, object_columns)
], remainder='passthrough').set_output(transform='pandas')


# Aplicando a transformação no DataFrame
df2 = preprocessing.fit_transform(df)



In [5]:
#df2['numeric__remainder__self_perception_weight']
df2['self_perception_overweight'] = df2['numeric__remainder__self_perception_weight'].apply(lambda x: True if x == 4 or x == 5 else False)
# Change the following line to drop the column by name
df2 = df2.drop(columns=['numeric__remainder__self_perception_weight'])
df2

,numeric__GPA__GPA,numeric__weight__weight,numeric__remainder__Gender,numeric__remainder__breakfast,numeric__remainder__calories_chicken,numeric__remainder__calories_day,numeric__remainder__calories_scone,numeric__remainder__coffee,numeric__remainder__comfort_food_reasons_coded,numeric__remainder__cook,...,object__remainder__eating_changes,object__remainder__father_profession,object__remainder__fav_cuisine,object__remainder__food_childhood,object__remainder__healthy_meal,object__remainder__ideal_diet,object__remainder__meals_dinner_friend,object__remainder__mother_profession,object__remainder__type_sports,self_perception_overweight
0,0.111111,187.00000,2.0,1.0,430.0,3.0,315.0,1.0,9.0,2.0,...,eat faster,profesor,arabic cuisine,rice and chicken,looks not oily,being healthy,"rice, chicken, soup",unemployed,car racing,False
1,0.807778,155.00000,1.0,1.0,610.0,3.0,420.0,2.0,1.0,3.0,...,i eat out more than usual.,self employed,italian,"chicken and biscuits, beef soup, baked beans","grains, veggies, (more of grains and veggies),...",try to eat 5-6 small meals a day. while trying...,"pasta, steak, chicken",nurse rn,basketball,False
2,0.611111,159.04918,1.0,1.0,720.0,4.0,420.0,2.0,1.0,1.0,...,sometimes choosing to eat fast food instead of...,owns business,italian,"mac and cheese, pizza, tacos",usually includes natural ingredients; nonproce...,i would say my ideal diet is my current diet,"chicken and rice with veggies, pasta, some kin...",owns business,none,False
3,0.555556,240.00000,1.0,1.0,430.0,3.0,420.0,2.0,2.0,2.0,...,"accepting cheap and premade,store bought foods",mechanic,turkish,"beef stroganoff, tacos, pizza","fresh fruits& vegetables, organic meats","healthy, fresh veggies,fruits & organic foods","grilled chicken ,stuffed shells,homemade chili",special education teacher,hockey,True
4,0.722222,190.00000,1.0,1.0,720.0,2.0,420.0,2.0,1.0,1.0,...,i have eaten generally the same foods but i do...,it,italian,"pasta, chicken tender, pizza","a lean protein such as grilled chicken, green ...",ideally i would like to be able to eat healthi...,"chicken parmesan, pulled pork, spaghetti and m...",substance abuse conselor,softball,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,0.722222,156.00000,1.0,1.0,610.0,4.0,420.0,2.0,2.0,3.0,...,i have noticed there is less time for a prepar...,accountant,italian,stromboli mac and cheese and pizza,mainly protein and vegetables with a complex c...,my ideal diet would consist of a majority of w...,"pasta, fish, steak",radiological technician,softball,True
121,0.444444,180.00000,1.0,1.0,265.0,2.0,315.0,2.0,2.0,3.0,...,eating pizza as an excuse when there is nothin...,doctor,mexican food,"isombe , plantains and ugali","a healthy meal is a variety of food , organic ...",eating home cooked meals everyday and being ab...,"fried rice ,baked potatoes ,curry chicken",public health advisor,basketball,True
122,0.934444,120.00000,1.0,1.0,720.0,3.0,420.0,1.0,2.0,3.0,...,less rice,ceo of company,korean,rice and potato,lots of vegetables,lots of veggies,"meat, rice, kimchi",real estate manageer,none,True
123,0.444444,135.00000,2.0,1.0,720.0,4.0,420.0,1.0,2.0,3.0,...,i don't eat as much on a daily basis since com...,store manager at giant eagle,italian,pizza and spaghetti,"a protein, a fruit, a starch, and a salad or s...",my ideal diet is the diet i am currently on. ...,"pizza, spaghetti, baked ziti",receptionist for a medical supply company,hockey,False


### Divisão do conjunto de dados

In [6]:
#Modelo de NB tratamento de Colunas não numericas (String)
def dropText(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df.drop(column, axis=1, inplace=True)
    return df

In [7]:
df2 = dropText(df2)

In [8]:
X_food = df2.drop(columns=['self_perception_overweight'], axis=1)
y_food = df2['self_perception_overweight']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_food, y_food, test_size=0.33, random_state=42)

### Seleção de algoritmos de classificação

 ## 1° Naive Bayes (NB) 
O algoritmo de Naïve Bayes no Scikit-learn é um método probabilístico, baseado no Teorema de Bayes, que assume independência entre as variáveis preditoras. Ele é eficiente para classificação de textos, filtragem de spam e outras tarefas. O Scikit-learn implementa variantes como GaussianNB, MultinomialNB e BernoulliNB.

 ## 2° k-Nearest Neighbors (kNN)
O k-Nearest Neighbors (kNN) é um algoritmo de aprendizado supervisionado usado para classificação e regressão. Ele funciona identificando os "k" vizinhos mais próximos de um dado ponto com base em uma métrica de distância, como a Euclidiana. A classificação é feita pela maioria dos votos entre os vizinhos, enquanto a regressão usa a média dos valores. Simples e eficiente, o kNN é ideal para problemas pequenos, mas pode ser computacionalmente caro em conjuntos de dados grandes. Ele é sensível à escala dos dados e à escolha do parâmetro "k".

 ## 3° Support Vector Machine (SVM)
O Support Vector Machine (SVM) é um algoritmo de aprendizado supervisionado usado para classificação e regressão, que busca encontrar um hiperplano ótimo que separe os dados em diferentes classes. Na versão linear, o SVM utiliza um hiperplano linear para separar os dados. Já com o kernel RBF (Radial Basis Function), ele mapeia os dados para um espaço de maior dimensão, permitindo a separação de padrões complexos de forma não linear. O SVM é eficaz em conjuntos de dados de alta dimensionalidade, mas requer uma boa escolha dos hiperparâmetros para evitar overfitting.

 ## 4° Decision Trees
As Decision Trees são modelos de aprendizado supervisionado usados para tarefas de classificação e regressão. Elas funcionam segmentando iterativamente os dados em subconjuntos baseados em perguntas ou condições sobre os atributos, formando uma estrutura hierárquica em forma de árvore. Cada nó interno representa uma decisão, cada ramo uma possível saída, e as folhas contêm as previsões finais. São intuitivas, fáceis de interpretar e lidam bem com dados mistos (numéricos e categóricos), mas podem sofrer de overfitting, especialmente em árvores muito profundas.

 ## 5° Random Forest  
O Random Forest é um algoritmo de aprendizado supervisionado baseado em conjuntos (ensemble), que combina múltiplas árvores de decisão para melhorar a precisão e reduzir o risco de overfitting. Ele constrói várias árvores independentes usando subconjuntos aleatórios de dados e atributos, e a previsão final é obtida pela média (regressão) ou pelo voto majoritário (classificação) das árvores. É robusto contra outliers e overfitting, além de lidar bem com dados complexos e de alta dimensionalidade. No entanto, pode ser computacionalmente intensivo para grandes conjuntos de dados.

 ## 6° Multilayer Perceptron 
O Multilayer Perceptron (MLP) é um tipo de rede neural artificial usado para aprendizado supervisionado, capaz de realizar tarefas de classificação e regressão. Ele é composto por pelo menos três camadas: entrada, ocultas (uma ou mais) e saída. As camadas ocultas contêm neurônios que usam funções de ativação não lineares, permitindo ao MLP modelar padrões complexos e não lineares nos dados. Treinado com algoritmos como backpropagation, o MLP é versátil, mas pode exigir muitos dados e ajuste fino dos hiperparâmetros para alcançar bom desempenho.

### Treinamento e avaliação

In [10]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Naive Bayes (NB)
nb_model = GaussianNB()

nb_model.fit(X_train,y_train)

# Avaliação inicial no conjunto de teste
y_pred = nb_model.predict(X_test)

# Calcular as métricas
accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted')  # Pode ser 'micro', 'macro', 'weighted' dependendo do seu problema
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Acurácia: 0.67
Precisão: 0.67
Recall: 0.67
F1-Score: 0.67


In [18]:
#K-Nearest Neighbor (kNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted')

recall = recall_score(y_test, y_pred, average='weighted')

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.60
Precisão: 0.56
Recall: 0.60
F1-score:0.58


In [12]:
#Support Vector Machines (SVM)
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

model = SVC(kernel='linear')

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted') 

recall = recall_score(y_test, y_pred, average='weighted')

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.60
Precisão: 0.62
Recall: 0.60
F1-score:0.61


In [13]:
#Decision Trees
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion='gini', random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted')  

recall = recall_score(y_test, y_pred, average='weighted')

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.74
Precisão: 0.73
Recall: 0.74
F1-score:0.73


In [14]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted')

recall = recall_score(y_test, y_pred, average='weighted')

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.74
Precisão: 0.72
Recall: 0.74
F1-score:0.68


In [15]:
#Multilayer Perceptron
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(5, 3), activation='relu', max_iter=1000, random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)

precision = precision_score(y_test, y_pred, average='weighted')

recall = recall_score(y_test, y_pred, average='weighted')

f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score:{f1:.2f}")

Acurácia: 0.62
Precisão: 0.49
Recall: 0.62
F1-score:0.55


# Avalaicao do desempenho de cada algoritmo no conjunto de teste usando métricas como acurácia, precisão, recall e F1-score.

| Algoritmo                             | Acurácia | Precisão | Recall | F1-Score |
|---------------------------------------|----------|----------|--------|----------|
| Naive Baye                           |    0.67  |    0.67  |  0.67  |    0.67  |
| k-Nearest Neighbors                  |    0.60  |    0.56  |  0.60  |    0.58  |
| Support Vector Machine (Linear/RBF)  |    0.60  |    0.62  |  0.60  |    0.61  |
| Decision Trees                       |    0.74  |    0.73  |  0.74  |    0.73  |
| Random Forest                        |    0.74  |    0.72  |  0.74  |    0.68  |
| Multilayer Perceptron                |    0.62  |    0.49  |  0.62  |    0.55  |


### Análise dos resultados

# Treinando os algoritmos com validação cruzada.

In [16]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

models = {
    "Naive Bayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "MLP": MLPClassifier(max_iter=2000,learning_rate_init=0.001)
}

# Funções para calcular as métricas de Precision, Recall e F1-score
scoring_metrics = {
    "Acurácia": "accuracy",
    "Precisão": make_scorer(precision_score, average='macro', zero_division=1),
    "Recall": make_scorer(recall_score, average='macro', zero_division=1),
    "F1-score": make_scorer(f1_score, average='macro', zero_division=1)
}

# Aplicação da validação cruzada
for name, model in models.items():
    print(f"Modelo: {name}")
    for metric_name, metric in scoring_metrics.items():
        scores = cross_val_score(model, X_food, y_food, cv=10, scoring=metric)
        print(f"{metric_name}: {scores.mean():.2f}")
    print('-' * 50)

Modelo: Naive Bayes
Acurácia: 0.73
Precisão: 0.68
Recall: 0.68
F1-score: 0.67
--------------------------------------------------
Modelo: k-NN
Acurácia: 0.58
Precisão: 0.43
Recall: 0.43
F1-score: 0.40
--------------------------------------------------
Modelo: SVM
Acurácia: 0.70
Precisão: 0.85
Recall: 0.50
F1-score: 0.41
--------------------------------------------------
Modelo: Decision Tree
Acurácia: 0.63
Precisão: 0.60
Recall: 0.58
F1-score: 0.59
--------------------------------------------------
Modelo: Random Forest
Acurácia: 0.71
Precisão: 0.81
Recall: 0.54
F1-score: 0.53
--------------------------------------------------
Modelo: MLP
Acurácia: 0.57
Precisão: 0.59
Recall: 0.51
F1-score: 0.46
--------------------------------------------------


| Algoritmo                             | Acurácia | Precisão | Recall | F1-Score |
|---------------------------------------|----------|----------|--------|----------|
| Naive Baye                           |    0.73  |    0.68  |  0.68  |    0.67  |
| k-Nearest Neighbors                  |    0.58  |    0.43  |  0.43  |    0.40  |
| Support Vector Machine (Linear/RBF)  |    0.70  |    0.85  |  0.50  |    0.41  |
| Decision Trees                       |    0.67  |    0.59  |  0.58  |    0.55  |
| Random Forest                        |    0.73  |    0.85  |  0.58  |    0.48  |
| Multilayer Perceptron                |    0.66  |    0.53  |  0.48  |    0.50  |


# Análise realizando ajuste de hiperparâmetros.

In [17]:
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

models = {
    "Naive Bayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(algorithm='auto',n_neighbors=9,weights='uniform'),
    "SVM": SVC(C=0.1, gamma='scale',kernel='rbf'),
    "Decision Tree": DecisionTreeClassifier(criterion='entropy',max_depth=3,min_samples_leaf=1,min_samples_split=5),
    "Random Forest": RandomForestClassifier(bootstrap=False,max_depth=10,min_samples_leaf=1,min_samples_split=5,n_estimators=100),
    "MLP": MLPClassifier(activation='relu',hidden_layer_sizes=(50,),max_iter=2000,learning_rate_init=0.1,solver='adam')
}

# Funções para calcular as métricas de Precision, Recall e F1-score
scoring_metrics = {
    "Acurácia": "accuracy",
    "Precisão": make_scorer(precision_score, average='macro', zero_division=1),
    "Recall": make_scorer(recall_score, average='macro', zero_division=1),
    "F1-score": make_scorer(f1_score, average='macro', zero_division=1)
}

# Aplicação da validação cruzada
for name, model in models.items():
    print(f"Modelo: {name}")
    for metric_name, metric in scoring_metrics.items():
        scores = cross_val_score(model, X_food, y_food, cv=10, scoring=metric)
        print(f"{metric_name}: {scores.mean():.2f}")
    print('-' * 50)

Modelo: Naive Bayes
Acurácia: 0.73
Precisão: 0.68
Recall: 0.68
F1-score: 0.67
--------------------------------------------------
Modelo: k-NN
Acurácia: 0.62
Precisão: 0.49
Recall: 0.44
F1-score: 0.38
--------------------------------------------------
Modelo: SVM
Acurácia: 0.70
Precisão: 0.85
Recall: 0.50
F1-score: 0.41
--------------------------------------------------
Modelo: Decision Tree
Acurácia: 0.70
Precisão: 0.70
Recall: 0.57
F1-score: 0.53
--------------------------------------------------
Modelo: Random Forest
Acurácia: 0.74
Precisão: 0.71
Recall: 0.57
F1-score: 0.57
--------------------------------------------------
Modelo: MLP
Acurácia: 0.70
Precisão: 0.85
Recall: 0.51
F1-score: 0.38
--------------------------------------------------


| Algoritmo                             | Acurácia | Precisão | Recall | F1-Score |
|---------------------------------------|----------|----------|--------|----------|
| Naive Baye                           |    0.73  |    0.68  |  0.68  |    0.67  |
| k-Nearest Neighbors                  |    0.62  |    0.49  |  0.44  |    0.38  |
| Support Vector Machine (Linear/RBF)  |    0.70  |    0.85  |  0.50  |    0.41  |
| Decision Trees                       |    0.70  |    0.71  |  0.57  |    0.55  |
| Random Forest                        |    0.75  |    0.75  |  0.60  |    0.59  |
| Multilayer Perceptron                |    0.70  |    0.85  |  0.53  |    0.40  |



# Resultados
| Algoritmo                             | Acurácia | Precisão | Recall | F1-Score |
|---------------------------------------|----------|----------|--------|----------|
| Naive Baye                           |    0.67  |    0.67  |  0.67  |    0.67  |
| k-Nearest Neighbors                  |    0.60  |    0.56  |  0.60  |    0.58  |
| Support Vector Machine (Linear/RBF)  |    0.60  |    0.62  |  0.60  |    0.61  |
| Decision Trees                       |    0.74  |    0.73  |  0.74  |    0.73  |
| Random Forest                        |    0.74  |    0.72  |  0.74  |    0.68  |
| Multilayer Perceptron                |    0.62  |    0.49  |  0.62  |    0.55  |
 
---

| Algoritmo                             | Acurácia | Precisão | Recall | F1-Score |
|---------------------------------------|----------|----------|--------|----------|
| Naive Baye                           |    0.73  |    0.68  |  0.68  |    0.67  |
| k-Nearest Neighbors                  |    0.58  |    0.43  |  0.43  |    0.40  |
| Support Vector Machine (Linear/RBF)  |    0.70  |    0.85  |  0.50  |    0.41  |
| Decision Trees                       |    0.67  |    0.59  |  0.58  |    0.55  |
| Random Forest                        |    0.73  |    0.85  |  0.58  |    0.48  |
| Multilayer Perceptron                |    0.66  |    0.53  |  0.48  |    0.50  |

---

| Algoritmo                             | Acurácia | Precisão | Recall | F1-Score |
|---------------------------------------|----------|----------|--------|----------|
| Naive Baye                           |    0.73  |    0.68  |  0.68  |    0.67  |
| k-Nearest Neighbors                  |    0.62  |    0.49  |  0.44  |    0.38  |
| Support Vector Machine (Linear/RBF)  |    0.70  |    0.85  |  0.50  |    0.41  |
| Decision Trees                       |    0.70  |    0.71  |  0.57  |    0.55  |
| Random Forest                        |    0.75  |    0.75  |  0.60  |    0.59  |
| Multilayer Perceptron                |    0.70  |    0.85  |  0.53  |    0.40  |





# Sem validacao cruzada
O Random Forest e as Decision Trees se destacaram com acurácia de 0.74, mas o Random Forest apresentou uma queda no F1-Score (0.68).
O Naive Bayes teve desempenho consistente (0.67 em todas as métricas).
SVM, k-NN e MLP apresentaram desempenho inferior, com acurácias abaixo de 0.63.

# Com validação cruzada
O desempenho geral caiu levemente, refletindo o impacto de uma avaliação mais rigorosa.
Random Forest e Naive Bayes ainda lideram em acurácia (0.73), enquanto o k-NN teve o pior desempenho geral, com acurácia de 0.58 e F1-Score de 0.40.
A SVM mostrou alta precisão (0.85), mas com baixo recall (0.50), indicando dificuldade em captar todas as classes positivas.

# Com ajuste de hiperparâmetros
Houve melhora geral para a maioria dos modelos, especialmente o Random Forest, que alcançou a melhor acurácia (0.75) e um equilíbrio razoável nas métricas.
A SVM manteve alta precisão (0.85), mas o recall permaneceu baixo (0.50), similar ao cenário anterior.
Naive Bayes continuou consistente, com resultados semelhantes aos obtidos na validação cruzada.
O k-NN mostrou uma pequena melhora em acurácia (0.62), mas ainda foi o algoritmo com desempenho mais fraco em termos gerais.

# Resultado final
O Random Forest destacou-se como o melhor modelo geral após o ajuste de hiperparâmetros, equilibrando as métricas. A SVM apresentou alta precisão, mas seu baixo recall pode limitar sua aplicabilidade em problemas sensíveis a falsos negativos. O Naive Bayes mostrou consistência ao longo de todos os cenários, sendo uma opção robusta para dados com características similares.